# Hyperparameter Tuning using HyperDrive



## Dataset

### Overview
I am using credit card transactions data from Kaggle ( more details on readme file). Problem statement is to predict whether the transaction is fraud (1) or not (0). This is a supervised binary classification task. 

In [1]:
from azureml.core import Workspace, Experiment
workspace = Workspace.from_config()
experiment_name = 'creditcardfraud'

experiment = Experiment(workspace, experiment_name)

print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: workspace-rainel
Azure region: brazilsouth
Subscription id: 611bccaf-ced7-4b1d-9395-57559c451c39
Resource group: raineldias88-rg


In [2]:
from azureml.core import Dataset

dataset = Dataset.get_by_name(workspace, name='creditcard')
df=dataset.to_pandas_dataframe()

In [3]:
# overview of data
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## Hyperdrive Configuration


In [4]:
# Make sure the compute cluster is set up
# check if compute clusters are set up
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "compute-cluster"
try:
    cpu_cluster = ComputeTarget(workspace=workspace, name=cpu_cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Explaination of termination policy, and param sampling

I have selected bandit policy for early termination. It defines an early termination policy based on slack criteria, and a frequency and delay interval for evaluation. Again the main reason for this policy selection is performance and saving resources. Any run that doesn't fall within the slack factor or slack amount of the evaluation metric with respect to the best performing run will be terminated, thus saving the compute resource. Concretely, the configuration used in my hyperdrive config will evaluate jobs every 1 step and will terminate jobs that are not within 10 percent slack of the best performing job at that particular step. On larger models, this strategy typically saves significant compute time with no impact on the performance of the best model trained.

In [5]:
import os

dependencies_dir = "./env"
os.makedirs(dependencies_dir, exist_ok=True)

In [6]:
%%writefile {dependencies_dir}/conda.yml
name: sklearn-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - pip:  
    - mlflow== 1.26.1
    - azureml-mlflow==1.42.0

Overwriting ./env/conda.yml


In [7]:
from azureml.core import Environment
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal


sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./env/conda.yml')

# Specify parameter sampler
ps = RandomParameterSampling({"--C": choice(0.1,1.0),
                              "--max_iter": choice(50,100)})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a estimator
est = ScriptRunConfig(source_directory=".",
                      compute_target=cpu_cluster,
                      script="train.py",
                      environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(run_config = est,
                                     hyperparameter_sampling = ps,                                    
                                     primary_metric_name = 'Accuracy',                                    
                                     max_total_runs = 20,
                                     max_concurrent_runs=4,   
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,  
                                     policy = policy)

In [8]:
from azureml.widgets import RunDetails
# Submit experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config)
RunDetails(hyperdrive_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [9]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_63c622c9-9227-423d-b0c1-40a21240420e
Web View: https://ml.azure.com/runs/HD_63c622c9-9227-423d-b0c1-40a21240420e?wsid=/subscriptions/611bccaf-ced7-4b1d-9395-57559c451c39/resourcegroups/raineldias88-rg/workspaces/workspace-rainel&tid=f2c80df2-0dbc-4eb5-9af1-fd0a90497ca6

Streaming azureml-logs/hyperdrive.txt

[2022-11-19T22:05:37.240031][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-11-19T22:05:38.0564126Z][SCHEDULER][INFO]Scheduling job, id='HD_63c622c9-9227-423d-b0c1-40a21240420e_0' 
[2022-11-19T22:05:38.1345949Z][SCHEDULER][INFO]Scheduling job, id='HD_63c622c9-9227-423d-b0c1-40a21240420e_1' 
[2022-11-19T22:05:38.3160676Z][SCHEDULER][INFO]Scheduling job, id='HD_63c622c9-9227-423d-b0c1-40a21240420e_2' 
[2022-11-19T22:05:38.378791][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-11-19T22:05:38.4154309Z][SCHEDULER][INFO]Scheduling job, id='HD_63c622c9-9227-423d-b0c1-40a21240420e_3' 
[2022-

{'runId': 'HD_63c622c9-9227-423d-b0c1-40a21240420e',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-11-19T22:05:36.17965Z',
 'endTimeUtc': '2022-11-19T22:16:12.233801Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '0fa09fc2-2581-4edb-9959-2dc197ef9aff',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': '4',
  'score': '0.9990637032875719',
  'best_child_run_id': 'HD_63c622c9-9227-423d-b0c1-40a21240420e_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_63c622c9-9227-423d-b0c1-40a21240420e_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClien

## Best Model



In [22]:
# get details of the best model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run ID : ',best_run.id)
print('\nMetrics: ',best_run_metrics)

print('\nParameters:', parameter_values)
print('\nAccuracy: ', best_run_metrics['Accuracy'])

Best Run ID :  HD_63c622c9-9227-423d-b0c1-40a21240420e_0

Metrics:  {'Regularization Strength:': 1.0, 'Max iterations:': 100, 'Accuracy': 0.9990637032875719}

Parameters: ['--C', '1', '--max_iter', '100']

Accuracy:  0.9990637032875719


In [23]:
print (best_run.log_confusion_matrix)

<bound method Run.log_confusion_matrix of Run(Experiment: creditcardfraud,
Id: HD_63c622c9-9227-423d-b0c1-40a21240420e_0,
Type: azureml.scriptrun,
Status: Completed)>


In [24]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
creditcardfraud,HD_63c622c9-9227-423d-b0c1-40a21240420e_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [30]:
#Save the best model
model= best_run.register_model(model_name='hyperdrive_best_model', model_path='./outputs',model_framework=Model.Framework.SCIKITLEARN,model_framework_version='0.19.1')




###  For deployment, AutoML is selected, so automl.ipynb file has the deployment code